<a href="https://colab.research.google.com/github/Kansi420/mobilenet_v2/blob/main/mobilenet_v2_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2

In [2]:
# Transformations for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 as MobileNetV2 expects this input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 45757439.70it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Load the pre-trained MobileNetV2 model
model = mobilenet_v2(pretrained=True)

# Modify the last layer to match the number of classes in CIFAR-10
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 135MB/s]


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [5]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
epochs = 5
# Training loop
for epoch in range(epochs):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

Epoch 1, Batch 200, Loss: 0.981
Epoch 1, Batch 400, Loss: 0.714
Epoch 1, Batch 600, Loss: 0.628
Epoch 1, Batch 800, Loss: 0.603
Epoch 1, Batch 1000, Loss: 0.570
Epoch 1, Batch 1200, Loss: 0.531
Epoch 1, Batch 1400, Loss: 0.519
Epoch 2, Batch 200, Loss: 0.440
Epoch 2, Batch 400, Loss: 0.425
Epoch 2, Batch 600, Loss: 0.436
Epoch 2, Batch 800, Loss: 0.447
Epoch 2, Batch 1000, Loss: 0.412
Epoch 2, Batch 1200, Loss: 0.420
Epoch 2, Batch 1400, Loss: 0.412
Epoch 3, Batch 200, Loss: 0.337
Epoch 3, Batch 400, Loss: 0.370
Epoch 3, Batch 600, Loss: 0.348
Epoch 3, Batch 800, Loss: 0.372
Epoch 3, Batch 1000, Loss: 0.367
Epoch 3, Batch 1200, Loss: 0.340
Epoch 3, Batch 1400, Loss: 0.377
Epoch 4, Batch 200, Loss: 0.290
Epoch 4, Batch 400, Loss: 0.326
Epoch 4, Batch 600, Loss: 0.317
Epoch 4, Batch 800, Loss: 0.324
Epoch 4, Batch 1000, Loss: 0.320
Epoch 4, Batch 1200, Loss: 0.314
Epoch 4, Batch 1400, Loss: 0.321
Epoch 5, Batch 200, Loss: 0.261
Epoch 5, Batch 400, Loss: 0.265
Epoch 5, Batch 600, Loss: 0.

KeyboardInterrupt: 

In [ ]:
##Interrupted training at 6th EPOCH

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 87.18%


In [7]:
print(device)

cuda
